<img src="https://www.th-koeln.de/img/logo.svg" style="float:right;" width="200">

# 13th exercise: <font color="#C70039">First Reinforcement Learning Q-Table learning</font>
* Course: AML
* Lecturer: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>
* Author of notebook: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>
* Author of notebook: <a href="https://www.gernotheisenberg.de/">Rubaiya Kabir Pranti</a>
* Matriculation Number: <a href="https://www.gernotheisenberg.de/"> 11146364</a>
* Date:   16.01.2024
---------------------------------
**GENERAL NOTE 1**: 
Please make sure you are reading the entire notebook, since it contains a lot of information on your tasks (e.g. regarding the set of certain paramaters or a specific computational trick), and the written mark downs as well as comments contain a lot of information on how things work together as a whole. 

**GENERAL NOTE 2**: 
* Please, when commenting source code, just use English language only. 
* When describing an observation please use English language, too.
* This applies to all exercises throughout this course.

---------------------------------

### <font color="FFC300">TASKS</font>:
The tasks that you need to work on within this notebook are always indicated below as bullet points. 
If a task is more challenging and consists of several steps, this is indicated as well. 
Make sure you have worked down the task list and commented your doings. 
This should be done by using markdown.<br> 
<font color=red>Make sure you don't forget to specify your name and your matriculation number in the notebook.</font>

**YOUR TASKS in this exercise are as follows**:
1. import the notebook to Google Colab or use your local machine.
2. make sure you specified you name and your matriculation number in the header below my name and date. 
    * set the date too and remove mine.
3. read the entire notebook carefully 
    * add comments whereever you feel it necessary for better understanding
    * run the notebook for the first time. 
4. play with all hypterparameters including the actions and states table.
5. add and implement an ϵ-greedy strategy 
---------------------------------

In [1]:
# Imports 
import numpy as np

### Create the possible states

In [2]:
# one dictionary location_to_state-> a physical location  'L1', 'L2'etc. maps to a numerical state 
# that the agent can understand and process
location_to_state = {
    'L1' : 0,
    'L2' : 1,
    'L3' : 2,
    'L4' : 3,
    'L5' : 4,
    'L6' : 5,
    'L7' : 6,
    'L8' : 7,
    'L9' : 8
}

state_to_location = dict((state,location) for location, state in location_to_state.items()) #states
#reversing

### Create the actions & rewards

In [3]:
actions = [0,1,2,3,4,5,6,7,8] #possible actions the agent can take
#actions = [0,1,2,3] #possible actions the agent can take

In [4]:
#rewards = np.array([[0,1,0,0],
                   #[1,0,1,0],
                   #[0,1,0,0],
                   #[0,0,0,0],
                   #[0,1,0,0],
                   #[0,0,1,0],
                   #[0,0,0,1],
                   #[0,0,0,0],
                   #[0,0,0,0]])

In [5]:
#rewards = np.array([[0,1,0,0,0,0,0,0,0],
                   #[1,0,1,0,1,0,0,0,0],
                   #[0,1,0,0,0,1,0,0,0],
                   #[0,0,0,0,0,0,1,0,0],
                   #[0,1,0,0,0,0,0,1,0],
                   #[0,0,1,0,0,0,0,0,0]])

In [6]:
rewards = np.array([[0,1,0,0,0,0,0,0,0],
                   [1,0,1,0,1,0,0,0,0],
                   [0,1,0,0,0,1,0,0,0],
                   [0,0,0,0,0,0,1,0,0],
                   [0,1,0,0,0,0,0,1,0],
                   [0,0,1,0,0,0,0,0,0],
                   [0,0,0,1,0,0,0,1,0],
                   [0,0,0,0,1,0,1,0,1],
                   [0,0,0,0,0,0,0,1,0]]) #represents the rewards tht the agent gets for moving from one state to another
# it encourages the agent to find best path possible with highest rewards from a starting location to an end location

### Def remaining hyperparameters

In [7]:
# initialize the parameters
gamma = 0.75 # discount factor
alpha = 0.9  # learning rate
# discount factor and learning rate tells how the future rewards are discounted 
# and how much new information overrides old information
epsilon = 1.0  # starting value for epsilon
min_epsilon = 0.01  # minimum value for epsilon
decay_rate = 0.001  # decay rate for epsilon per episode

### Define agent and its attributes

In [8]:
# class QAgent is defined with methods to perform the training
# and to find the optimal route (get_optimal_route) based on the Q-table it learns

class QAgent1():
    # initialize everything
    def __init__(self, alpha, gamma, location_to_state, actions, rewards, state_to_location):
        self.gamma = gamma
        self.alpha = alpha
        
        self.location_to_state = location_to_state
        self.actions = actions
        self.rewards = rewards
        self.state_to_location = state_to_location
        
        # remember, the Q-value table is of size all actions x all states
        M = len(location_to_state)
        self.Q = np.zeros((M,M), dtype = None, order = 'C')
        
    # now, implement the training method for the agent
    def training(self, start_location, end_location, iterations):

        rewards_new = np.copy(self.rewards)

        ending_state = self.location_to_state[end_location]
        
        rewards_new[ending_state, ending_state] = 999
        
        # DEBUG
        print(rewards_new)

        # pick random current state
        # iterations = the # of training cycles
        for i in range(iterations):
            current_state = np.random.randint(0,9)
            playable_actions = []

            # iterate thru the rewards matrix to get states
            # that are really reachable from the randomly chosen
            # state and assign only those in a list since they are really playable
            for j in range(9):
                if rewards_new[current_state, j] > 0:
                    playable_actions.append(j)

            # choosing next random state
            # however, make sure that playable_actions is not empty
            if len(playable_actions) != 0:
                # Randomly select a value from playable_actions
                next_state = np.random.choice(playable_actions)
    
            # finding the difference in Q, often referred to as temporal difference
            # by means of the Bellman's equation
            TD = rewards_new[current_state, next_state] + self.gamma * self.Q[next_state, np.argmax(self.Q[next_state,])] - self.Q[current_state, next_state]
            # combine with the learning rate
            self.Q[current_state, next_state] += self.alpha*TD
            # DEBUG
            #print(f"Q[{current_state}, {next_state}]:", self.Q[current_state, next_state])

        route = [start_location]
        next_location = start_location

        # print the optimal route from start to end
        self.get_optimal_route(start_location, end_location, next_location, route, self.Q)

    # compute the optimal route
    def get_optimal_route(self, start_location, end_location, next_location, route, Q):
        while(next_location != end_location):
            starting_state = self.location_to_state[start_location]
            next_state = np.argmax(Q[starting_state,])
            next_location = self.state_to_location[next_state]
            route.append(next_location)
            start_location = next_location
        # DEBUG
        print('Q-table:',Q)
        print("optimal route:", route)

In [9]:
qagent1 = QAgent1(alpha, gamma, location_to_state, actions, rewards, state_to_location)
# is creating an instance of a QAgent class with specific parameters 
qagent1.training('L9', 'L1', 1000)

[[999   1   0   0   0   0   0   0   0]
 [  1   0   1   0   1   0   0   0   0]
 [  0   1   0   0   0   1   0   0   0]
 [  0   0   0   0   0   0   1   0   0]
 [  0   1   0   0   0   0   0   1   0]
 [  0   0   1   0   0   0   0   0   0]
 [  0   0   0   1   0   0   0   1   0]
 [  0   0   0   0   1   0   1   0   1]
 [  0   0   0   0   0   0   0   1   0]]
Q-table: [[3995.99804196 2249.4949161     0.            0.            0.
     0.            0.            0.            0.        ]
 [2997.99800052    0.         1688.12118684    0.         1688.12118579
     0.            0.            0.            0.        ]
 [   0.         2249.49491647    0.            0.            0.
  1267.09089047    0.            0.            0.        ]
 [   0.            0.            0.            0.            0.
     0.          951.31813021    0.            0.        ]
 [   0.         2249.49491647    0.            0.            0.
     0.            0.         1267.09088187    0.        ]
 [   0.         

# with epsilon-greedy strategy

In [10]:
# adding an epsilon parameter to the class and modify the training method

class QAgent2():
    # initialize everything, adding epsilon parameter
    def __init__(self, alpha, gamma, epsilon, location_to_state, actions, rewards, state_to_location):
        self.gamma = gamma
        self.alpha = alpha
        self.epsilon = epsilon  #epsilon for the epsilon-greedy strategy
        
        self.location_to_state = location_to_state
        self.actions = actions
        self.rewards = rewards
        self.state_to_location = state_to_location
        
        M = len(location_to_state)
        self.Q = np.zeros((M, M), dtype=float, order='C')
        
    def any_action_chosen(self, current_state, play_actions):
        # implement epsilon-greedy strategy
        if np.random.uniform(0, 1) < self.epsilon:
            # exploration>>>by chooseing a random action
            next_state = np.random.choice(play_actions)
        else:
            # exploitation>>> choosing the best action from Q-table if available otherwise random action
            current_q_values = self.Q[current_state, play_actions]
            max_q_value = np.max(current_q_values)
            if max_q_value == 0:
                # If all Q-values are zero, choosing randomly among the playable actions==play_actions
                next_state = np.random.choice(play_actions)
            else:
                # selecting the action with the highest Q-value
                max_q_actions = np.where(current_q_values == max_q_value)[0]
                next_state = np.random.choice(max_q_actions)
        return next_state

    # now, implementing the training method for the agent
    def training(self, start_location, end_location, iterations, epsilon_decay):
        rewards_new = np.copy(self.rewards)
        ending_state = self.location_to_state[end_location]
        rewards_new[ending_state, ending_state] = 999

        for i in range(iterations):
            current_state = np.random.randint(0, len(self.actions))
            play_actions = [j for j in range(len(self.actions)) if rewards_new[current_state, j] > 0]

            if play_actions:
                next_state = self.any_action_chosen(current_state, play_actions)
                # finding the difference in Q> often referred to as temporal difference
                # by means of *******the Bellman's equation*********
                TD = rewards_new[current_state, next_state] + self.gamma * self.Q[next_state, np.argmax(self.Q[next_state,])] - self.Q[current_state, next_state]
                # combine with the learning rate
                self.Q[current_state, next_state] += self.alpha * TD

            # updating epsilon with decay
            self.epsilon = max(self.epsilon - epsilon_decay, 0.01)  # ensuring epsilon does not go below 0.01

        # after training, using the learned Q-table to determine the optimal path
        # print the optimal route from start to end
        return self.get_optimal_route(start_location, end_location)
    
    # compute the optimal route
    def get_optimal_route(self, start_location, end_location):
        route = [start_location]
        next_location = start_location
        while next_location != end_location:
            starting_state = self.location_to_state[start_location]
            next_state = np.argmax(self.Q[starting_state,])
            next_location = self.state_to_location[next_state]
            route.append(next_location)
            start_location = next_location
        return route
        #print('Q-table:',Q)
        #print("optimal route:", route)

In [11]:
# with  epsilon-greedy strategy
qagent2 = QAgent2(alpha, gamma, epsilon, location_to_state, actions, rewards, state_to_location)
# is creating an instance of a QAgent class with specific parameters 
qagent2.training('L9', 'L1', 1000, 0.005)

['L9', 'L1']

### <font color="FFC300">Comments</font>:
#### - Through epsilon-greedy strategy, now the optimal path is more straighforward because agent has already known through exploitation which will be the best action to get maximum cumulative reward 
#### - Even if i had increased the epsilon and decreased decay rate values for pushing for more exploration in the environment, the optimal path is same like before, directly from L9 to L1
#### - if environment does not allow such path from L9 to L1 directly, it is illogical 
#### - but it would only be realistic if such a direct transition is possible and rewarding in our defined environment

### <font color="FFC300">Comments</font>:
#### -- In this context, the agent is supposed to learn the best actions to take in our given environment to maximize cumulative reward

#### -- when the agent starts from state L9 and aims to reach the end state L1> the optimal path will depend on the Q-values which are a result of the given reward matrix and the exploration( the agent experinecs during training). If some paths have a reward of zero, the agent may still cross those paths during the exploration phase only, but it will learn over time to give priority to paths that yield higher cumulative rewards

#### -- epsilon-greedy strategy is a common approach used in Q-learning to balance exploration and exploitation

#### -- with above strategy, the agent will mostly choose the best-known action known as exploitation but occasionally can choose a random action>>exploration. A generl approach is to start with a higher epsilon==more exploration and gradually it will be decreased as the agent learns more about the environment

#### -- zeros typically indicate that there is no value in taking that action from that state because it doesn't lead to an improvement in the long-term cumulative reward

#### -- In the training method, the agent undergoes a number of iterations where it updates its Q-values based on the Bellman equation by using the alpha and gamma hyperparameters

#### -- After training, the get_optimal_route method is used to determine the best path from a starting to an ending using the learned Q-values

#### -- The discount factor quantifies how much importance is given to future rewards

#### -- when gamma = 0.75, alpha = 0.9, then the optimal route is ['L9', 'L8', 'L5', 'L2', 'L1'] for qagent.training('L9', 'L1', 1000)
#### -- when gamma = 0.9, alpha = 0.9(constant), q values are increased in some states than before as importance is given to future rewards
#### -- when gamma = 0.2, alpha = 0.9(constant), q values are decreased in those states than before as importance is not given to future rewards
#### -- when gamma = 0.75(constant), alpha = 0.1, q values are decreased in those states than before as the new information did not override those old information at a great extent
#### -- when gamma = 0.75(constant), alpha = 0.5, q values are almost like the value achieved when alpha = 0.9
#### -- if actions has been limitd to [0,1,2,3] only, the optimal path will be same like epsilon greedy strategy which is turned to optimal route: ['L9', 'L1']